In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# make a new directory for new images
# !mkdir /content/drive/MyDrive/PyPSA_Africa_images/maxar/australia

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=5f982c7f99d14a52e6e8d2932856ba74a2750f243e9419f8edd9b4f29ab3cc0d
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import os
import wget

# move to respective dir
# os.chdir('/content/drive/MyDrive/PyPSA_Africa_images/maxar/south_africa/')
# os.mkdir('/content/drive/MyDrive/PyPSA_Africa_images/maxar/south_africa/raw/')

In [ ]:
from tqdm import tqdm

path = '/content/drive/MyDrive/PyPSA_Africa_images/maxar/philippines/'
if not os.path.isdir(os.path.join(path, 'raw')): os.mkdir(os.path.join(path, 'raw'))
os.chdir(os.path.join(path, 'raw'))
file = os.path.join(path, 'philippines.txt')

print(f'currently working in directory: {os.getcwd()}')



currently working in directory: /content/drive/MyDrive/PyPSA_Africa_images/maxar/philippines/raw


In [ ]:
with open(file, 'r') as f:
    for line in tqdm(f.readlines()):
        if not 'vegetation' in line:
            wget.download('https://' + line.split('//')[-1])
            # print(line)
            # wget.download(line)

100%|█████████▉| 2552/2553 [39:03<00:00,  1.09it/s]


URLError: ignored

In [24]:
!pip install geopandas
!pip install rtree
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 1.4 MB/s 


In [ ]:
from PIL import Image
from osgeo import osr, gdal
import numpy as np
from gdalconst import *
from shapely.geometry import Polygon
import geopandas as gpd
import folium
import os
import rtree
import matplotlib.pyplot as plt
from tqdm import tqdm
# import pygeos

# os.chdir(os.path.dirname(os.path.abspath(__file__)))  # move up to parent directory

#%%
def show_coverage(path, resume_file=None):
    """
    Creates GeoDataFrame, checks the tif files in a desired dir 
    and adds their geometries (with respective file name) as row
    to GeoDataFrame
    -----------
    Arguments:
    path : (str)
        directory with tif files
    """

    tif_files = []

    # iterate over all .tif files in desired dir
    for filename in os.listdir(path):
        if filename.endswith(".tif"): 
            tif_files.append(os.path.join(path, filename))

    # add data to existing dataframe if desired
    try:
        coverage = gpd.read_file(resume_file)
        print(f"Adding coverage to {resume_file}")
    except:
        coverage = gpd.GeoDataFrame({"filename": [], "geometry": []})

    # extract geometries of .tif files and add to geodataframe
    print('Obtaining geometries...')

    for file in tqdm(tif_files):
        # try:
        #     info = gdal.Info(file, format="json")
        # except SystemError:
        #     print(f'Skipping raster {file}')
        #     continue

        info = gdal.Info(file, format="json")

        # corners = info["cornerCoordinates"]
        corners = info["wgs84Extent"]['coordinates'][0]
        poly = Polygon((
                    # corners["upperLeft"],
                    # corners["upperRight"],
                    # corners["lowerRight"],
                    # corners["lowerLeft"]
                    corners[0],
                    corners[1],
                    corners[2],
                    corners[3]
                        ))
        coverage = coverage.append({"filename": file, "geometry": poly}, ignore_index=True)


    coverage = coverage.set_crs(epsg=4326)

    return coverage

In [ ]:
path = '/content/drive/MyDrive/PyPSA_Africa_images/maxar/drc/raw/'

gdf = show_coverage(path)

Obtaining geometries...


100%|██████████| 218/218 [03:11<00:00,  1.14it/s]


In [ ]:
gdf.head()

,filename,geometry
0,/content/drive/MyDrive/PyPSA_Africa_images/max...,"POLYGON ((29.06540 -3.27143, 29.06540 -3.47519..."
1,/content/drive/MyDrive/PyPSA_Africa_images/max...,"POLYGON ((29.07814 -3.27776, 29.07814 -3.47073..."
2,/content/drive/MyDrive/PyPSA_Africa_images/max...,"POLYGON ((28.83041 -2.42220, 28.83041 -2.57517..."
3,/content/drive/MyDrive/PyPSA_Africa_images/max...,"POLYGON ((28.83801 -2.43146, 28.83801 -2.56502..."
4,/content/drive/MyDrive/PyPSA_Africa_images/max...,"POLYGON ((29.13427 -2.98796, 29.13427 -4.01239..."


In [ ]:
import folium 

m = folium.Map(zoom_start=10, tiles='CartoDB positron')

for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['filename'].split('/')[-1]).add_to(geo_j)
    geo_j.add_to(m)
m

In [27]:
import rasterio as rio
from shapely.geometry import box

url = 'https://opendata.digitalglobe.com/events/covid19-malawi/pre-event/2019-03-15/103001008C1DAE00/103001008C1DAE00.tif'

def get_url_bounds(url):
  with rio.open(url) as cov:
    bounds = cov.bounds
    geom = box(*bounds)
    return geom

x = get_url_bounds(url)

print(x)

POLYGON ((33.997457452020484 -11.602679506407842, 33.997457452020484 -11.104872126899522, 33.815188281946035 -11.104872126899522, 33.815188281946035 -11.602679506407842, 33.997457452020484 -11.602679506407842))
